In [34]:
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from PIL import Image
import imutils
from openpyxl import Workbook
import os, os.path
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
#Function for displaying an image
def show(show_img):
    cv2.imshow("Image", show_img) 
    cv2.waitKey(0)

#Function for getting the x,y,w,h values for ROI
def get_crdnt(image):
    r = cv2.selectROI("Image", image)
    (x,y,w,h) = r
    # Crop image
    imCrop = image[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    # Display cropped image
    show(imCrop)
    print("x",x,"y",y, "w", w,"h",h )

#Function for cropping an image based on x,y,w,h values
def crop_road(x,y,w,h,image_full):
    r = (x,y,w,h)
    imCrop = image_full[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    return imCrop 

#Function for reading an image from folder and then cropping an image
def get_image(filepath,filename):  
    image = cv2.imread(filepath+filename)
    image = cv2.resize(image, (960, 540)) 
    cropped = crop_road(4,122,956,244,image)
    return cropped
    

In [3]:
#Create a pandas dataframe for 244x956 = 233265 pixels
col = [x for x in range(233264)]
col = ["label"] + col
road_pixels = pd.DataFrame(columns=col)
road_pixels

,label,0,1,2,3,4,5,6,7,8,...,233254,233255,233256,233257,233258,233259,233260,233261,233262,233263


In [4]:
z = 0
#path to your folder where all the categories of road are present
PATH = r"./Data/"
#read all folders in PATH
paths = os.listdir(PATH)
paths.remove('.ipynb_checkpoints')

#inside each folder read each image
for path in paths:
    FPATH = r"./Data/"
    FILE_PATH = FPATH + str(path) + "/"
    filenames = os.listdir(FILE_PATH)

    cat = re.sub('Cat', '', path)

# for each image perform image preprocessing 
    for filename in filenames:
        print(FILE_PATH + filename)
        
        if filename == ".ipynb_checkpoints":
            continue
        
        road = get_image(FILE_PATH,filename)
        img = road
#         show(img)

        #convert BGR to HSV
        imgHSV= cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

#         show(imgHSV)
        #get mean and standard deviation of pixel colour values
        means, stddevs  = cv2.meanStdDev(imgHSV)
        
        # Set lowest value of colur
        lowerBound= means - stddevs
        # Set highest value of colur
        upperBound= means + stddevs
         
        #kernel of 5x5 matrix of ones
        kernelOpen=np.ones((5,5))
        #kernel of 20x20 matrix of ones
        kernelClose=np.ones((20,20))

        # create the Mask
        mask=cv2.inRange(imgHSV,lowerBound,upperBound)
        #morphology
        maskOpen=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernelOpen)
        maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)
        #Use maskClose
        maskFinal=maskClose
        kernel = np.ones((5,5),np.uint8)
        erosion = cv2.erode(maskFinal,kernel,iterations = 7)
         
        #get contours
        conts,h=cv2.findContours(erosion.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)



        hull = []

        # calculate hull for each contour
        for i in range(len(conts)):
            # creating convex hull object for each contour
            hull.append(cv2.convexHull(conts[i], False))

        # draw contours and hull points
        # for i in range(len(conts)):
        #     #cv2.drawContours(img,conts,-1,(0,0,255),3)
        #     # draw ith convex hull object
        #     cv2.drawContours(img, hull, i, (0,0,255), 1, 8)

        
        #create matrix with a shape of img.shape
        stencil = np.zeros(img.shape).astype(img.dtype)
        
        #set color black for areas outside hull
        color = [255, 255, 255]
        cv2.fillPoly(stencil, hull, color)
        result = cv2.bitwise_and(img, stencil)


#         show(result)
        # convert the processed image into Grayscale
        resultGray= cv2.cvtColor(result,cv2.COLOR_BGR2GRAY)
        block_size = 293
        constant = 2
        #Threshold converted image
        th1 = cv2.adaptiveThreshold(resultGray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, block_size, constant)
        th2 = cv2.adaptiveThreshold (resultGray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, constant)
#         show(th2)
        
        #convert the numpy array of image pixel data to list
        row = th2.ravel()
        row_as_list = row.tolist()
        
        #attach category type for each imagevb
        row_as_list.insert(0,cat)
        row_as_list
        
        #convert the lists to pandas dataframe
        road_pixels.loc[z] = row_as_list

        z+=1



./Data/Cat1/IMG_20191006_114129.jpg
./Data/Cat1/PASER-Fail-1.jpg
./Data/Cat1/PASER-Fail-2.jpg
./Data/Cat10/IMG_20191006_112008.jpg
./Data/Cat10/IMG_20191006_112010.jpg
./Data/Cat10/IMG_20191006_112012.jpg
./Data/Cat2/IMG_20191006_113217.jpg
./Data/Cat2/IMG_20191006_113227.jpg
./Data/Cat2/IMG_20191006_113227_1.jpg
./Data/Cat2/IMG_20191006_113229.jpg
./Data/Cat2/IMG_20191006_114039.jpg
./Data/Cat2/IMG_20191006_114128.jpg
./Data/Cat2/IMG_20191006_114132.jpg
./Data/Cat2/PASER-VeryPoor1.jpg
./Data/Cat2/PASER-VeryPoor2.jpg
./Data/Cat3/IMG_20191006_112615.jpg
./Data/Cat3/PASER-Poor1.jpg
./Data/Cat3/PASER-Poor2.jpg
./Data/Cat4/IMG_20190811_151005.jpg
./Data/Cat4/IMG_20190811_151032.jpg
./Data/Cat4/IMG_20191006_112616.jpg
./Data/Cat4/IMG_20191006_114056_1.jpg
./Data/Cat4/IMG_20191006_114102.jpg
./Data/Cat4/IMG_20191006_114103.jpg
./Data/Cat4/IMG_20191006_114107.jpg
./Data/Cat4/IMG_20191006_114112.jpg
./Data/Cat4/IMG_20191006_114121.jpg
./Data/Cat4/PASER-Fair1.jpg
./Data/Cat4/PASER-Fair2.jpg
./D

In [5]:
road_pixels = road_pixels.reset_index(drop=True)
road_pixels

,label,0,1,2,3,4,5,6,7,8,...,233254,233255,233256,233257,233258,233259,233260,233261,233262,233263
0,1,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,255,255,255,255,255
1,1,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,255
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10,0,0,0,0,0,0,0,0,0,...,0,0,255,255,0,255,255,255,255,0
4,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,10,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,255,255,255,255,255
6,2,255,255,255,255,255,255,255,255,255,...,255,0,0,0,0,0,0,0,0,0
7,2,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,255,255,255,255,255
8,2,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,255,255,255,255,255
9,2,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,255,255,255,255,255


In [12]:
#X_train
X_train = road_pixels.drop('label', 1).to_numpy().reshape(len(road_pixels),956,244,1).astype('float32')
X_train /= 255

#y_train
y_train = road_pixels["label"]

y_train = y_train.values.astype('float32')

y_train = np.array(y_train)

y_train = to_categorical(y_train)

print(y_train)

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.

In [1]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(2,2), input_shape=(956, 244,1), padding='same', activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(64, kernel_size=(2,2), padding='same', activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(128, kernel_size=(2,2), padding='same', activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(64, kernel_size=(2,2), padding='same', activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(32, kernel_size=(2,2), padding='same', activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dense(11, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

NameError: name 'Sequential' is not defined

In [14]:
history = model.fit(X_train, y_train, epochs=20, validation_split=0.3)

Train on 46 samples, validate on 20 samples
Epoch 1/20
46/46 [==============================] - 51s 1s/sample - loss: 2.4239 - acc: 0.0652 - val_loss: 3.3595 - val_acc: 0.0000e+00
Epoch 2/20
46/46 [==============================] - 9s 199ms/sample - loss: 1.9868 - acc: 0.1957 - val_loss: 3.7953 - val_acc: 0.0000e+00
Epoch 3/20
46/46 [==============================] - 21s 459ms/sample - loss: 1.9234 - acc: 0.1957 - val_loss: 4.5690 - val_acc: 0.0000e+00
Epoch 4/20
46/46 [==============================] - 9s 190ms/sample - loss: 1.8508 - acc: 0.3696 - val_loss: 5.7754 - val_acc: 0.0000e+00
Epoch 5/20
46/46 [==============================] - 8s 185ms/sample - loss: 1.6804 - acc: 0.3913 - val_loss: 6.4005 - val_acc: 0.0000e+00
Epoch 6/20
46/46 [==============================] - 9s 188ms/sample - loss: 1.6416 - acc: 0.4130 - val_loss: 6.4288 - val_acc: 0.0000e+00
Epoch 7/20
46/46 [==============================] - 9s 193ms/sample - loss: 1.4404 - acc: 0.5870 - val_loss: 6.9259 - val_acc: 0.

In [30]:
model.save("trained_model.h5")

In [15]:
#model accuracy
score_trainset = model.evaluate(X_train, y_train)

66/66 [==============================] - 2s 38ms/sample - loss: 11.6594 - acc: 0.6970


In [16]:
#predict for train set
predicted_categories = model.predict(X_train)
predicted_categories = np.argmax(np.round(predicted_categories),axis=1)

In [17]:
#predictaed labels for train set
predicted_categories

array([ 1,  1,  1, 10, 10, 10,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,
        3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  0,  5,  0,  6, 10,
       10,  5,  2,  5, 10,  6,  5,  5,  4, 10,  2,  2, 10,  4,  2],
      dtype=int64)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


66/66 [==============================] - 3s 46ms/sample - loss: 11.6594 - acc: 0.6970
